In [ ]:
# домашняя работа 

In [ ]:
# 1. Теперь решаем задачу регрессии - предскажем цены на недвижимость. 
#    Использовать датасет 
#    https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
# 2. Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию 
#    для оценки качества моделей
# 3. Построить случайный лес, вывести важность признаков
# 4. Обучить стекинг как минимум 3х моделей, использовать 
#    хотя бы 1 линейную модель и 1 нелинейную
# 5. Для валидации модели 2-го уровня использовать 
#    отдельный hold-out датасет, как на занятии
# 6. Показать, что использование ансамблей моделей действительно 
#    улучшает качество (стекинг vs другие модели сравнивать на hold-out)
# 7. В качестве решения: Jupyter notebook с кодом, комментариями 
#    и графиками, ссылка на гитхаб


In [21]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('train.csv')
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non

In [14]:
# Закодировать nominal категориальную переменную с помощью "one hot encoding"  (используем реализацию из лекций)
def get_one_hot(df, cols):
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
    return df 

# Закодировать ordered категориальную переменную используя числовой порядок
def get_ordered_col(dataset, col, dict_col_params):
    """
    В функцию передается датасет, имя колонки и словарь {'значение_параметра_в_колонке': числовое_значение_для_замены }
    """
    dataset[col + '_order'] = dataset.apply(lambda x: dict_col_params[x[col]],axis=1)
    return dataset

def get_ordered_cols(dataset, cols, dict_col_params):
    for col in cols:
        res = get_ordered_col(dataset, col, dict_col_params)
    return res.drop(columns=cols)     

dict_label_5_1 = {'Ex': 4,'Gd': 3,'TA': 2,'Fa': 1,'Po': 0}
dict_label_5_2 = {'Ex': 4,'Gd': 3,'TA': 2,'Fa': 1,'NA': 0}
dict_label_5_3 = {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'NA': 0} 
dict_label_6_1 = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}
dict_label_7_1 = {'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'NA': 0} 


In [34]:
# функция подготовки данных
def get_data(dataset):
    data_raw = dataset.copy()
    data_raw = get_one_hot(dataset, ['MiscFeature', 'Fence', 'Alley', 'Street', 'LotShape', 'LandContour', 'Utilities',\
                                'LotConfig', 'LandSlope', 'BldgType', 'MasVnrType', 'Foundation', 'Heating', \
                                'CentralAir', 'Electrical', 'PavedDrive', 'MSZoning','MSSubClass','Neighborhood','Condition1',\
                                       'Condition2','HouseStyle','RoofStyle','RoofMatl','Exterior1st',\
                                       'Exterior2nd','Functional','GarageType','GarageFinish',\
                                       'SaleType','SaleCondition'])
    
    data_raw = get_ordered_cols(data_raw, ['ExterQual', 'ExterCond', 'HeatingQC', 'KitchenQual'], dict_label_5_1)
    data_raw = get_ordered_cols(data_raw, ['PoolQC'], dict_label_5_2)
    data_raw = get_ordered_cols(data_raw, ['BsmtExposure'], dict_label_5_3)
    data_raw = get_ordered_cols(data_raw, ['BsmtQual', 'BsmtCond', 'FireplaceQu', 'GarageQual', 'GarageCond'] , dict_label_6_1)
    data_raw = get_ordered_cols(data_raw, ['BsmtFinType1', 'BsmtFinType2'] , dict_label_7_1) 
    
    y = data_raw['SalePrice'].copy()
    #data_raw= data_raw.drop(columns=['SaleCategory'])
    columns_to_drop = ['SalePrice', 'MSZoning','MSSubClass','Neighborhood','Condition1',\
                                  'Condition2','HouseStyle','RoofStyle','RoofMatl','Exterior1st',\
                                  'Exterior2nd','Functional','GarageType','GarageFinish',\
                                  'SaleType','SaleCondition','MiscFeature', 'Fence', 'Alley',\
                                  'Street', 'LotShape', 'LandContour', 'Utilities',\
                                  'LotConfig', 'LandSlope', 'BldgType', 'MasVnrType', \
                                  'Foundation', 'Heating', 'CentralAir', 'Electrical', 'PavedDrive']
    
    data_raw = data_raw.drop(columns=columns_to_drop)
    
        
    
    return data_raw, y
    

In [35]:
X_full, y_full = get_data(data)

In [36]:
X_full.head()

,Id,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,KitchenQual_order,PoolQC_order,BsmtExposure_order,BsmtQual_order,BsmtCond_order,FireplaceQu_order,GarageQual_order,GarageCond_order,BsmtFinType1_order,BsmtFinType2_order
0,1,65.0,8450,7,5,2003,2003,196.0,706,0,...,3,0,1,4,3,0,3,3,6,1
1,2,80.0,9600,6,8,1976,1976,0.0,978,0,...,2,0,4,4,3,3,3,3,5,1
2,3,68.0,11250,7,5,2001,2002,162.0,486,0,...,3,0,2,4,3,3,3,3,6,1
3,4,60.0,9550,7,5,1915,1970,0.0,216,0,...,3,0,1,3,4,4,3,3,5,1
4,5,84.0,14260,8,5,2000,2000,350.0,655,0,...,3,0,3,4,3,3,3,3,6,1


In [37]:
# X и y общая выборка для моделей 1 уровня 
# X_out,y_out - выборка для проверки модели второго уровня

X,y,X_out,y_out = train_test_split(X_full, y, test_size=0.1 ,random_state=42)